In [1]:
import dabench as dab
import numpy as np
import jax
from timeit import default_timer as timer
import pandas as pd
import jaxlib

from ray import train, tune
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch

# Define parameters

In [2]:
year_in_timesteps = 4380
spinup_size = 5*year_in_timesteps
valid_size = round(year_in_timesteps/4)
transient_size = 1*year_in_timesteps
test_size = 1*year_in_timesteps

In [3]:
nr_steps = spinup_size + valid_size + transient_size + test_size
delta_t=7200
analysis_window = 6*delta_t
analysis_time_in_window = 3*delta_t
num_epochs = 3

### Function definition: Backprop 4DVar

We'll need to prep and run Backprop-4DVar many times, so this wraps it all into functions.

Note: Since we're using raytune, we're separating this into a prep function and a run function to speed things up

In [4]:
def prep_backprop_4dvar(system_dim_xy, nr_steps, spinup_size, valid_size, test_size,
                        test_run, delta_t, sigma_bg_multiplier, sigma_obs_multiplier, 
                        random_seed):
    
    np_rng = np.random.default_rng(random_seed)
    jax.clear_backends()

    
    ### Nature Run
    nature_run = dab.data.PyQGJax(nx=system_dim_xy, ny=system_dim_xy, delta_t=delta_t, 
                                  store_as_jax=True, random_seed=random_seed)

    nature_run.generate(n_steps=nr_steps) 
    nr_spinup, nr_valid, nr_transient_and_test = nature_run.split_train_valid_test(
        spinup_size, valid_size, transient_size + test_size)
    nr_transient, nr_test, _ = nr_transient_and_test.split_train_valid_test(
        transient_size, test_size, 0)

    if not test_run:
        nr_eval = nr_valid
    else:
        nr_eval = nr_test
        
        
    ### Observations
    obs_location_count = round(nature_run.system_dim/2)

    # First we need to calculate the per-variable SD for QGS model
    obs_sd_scale = 0.1
    per_variable_sd = np.std(nr_spinup.values, axis=0)
    obs_sd = 0.1*per_variable_sd

    obs_pyqg = dab.observer.Observer(
        nr_eval,
        time_indices = np.arange(0, nr_eval.time_dim, 3),
        random_location_count = obs_location_count,
        error_bias = 0.0,
        error_sd = obs_sd,
        random_seed=random_seed+test_run,
        stationary_observers=True,
        store_as_jax=True
    )

    obs_vec_pyqg = obs_pyqg.observe()

    
    ### Forecast Model
    model_pyqg = dab.data.PyQGJax(nx=system_dim_xy, ny=system_dim_xy,
                                  store_as_jax=True, random_seed=random_seed)

    class PyQGModel(dab.model.Model):                                                                       
        """Defines model wrapper for Lorenz96 to test forecasting."""
        def forecast(self, state_vec, n_steps):
            gridded_values = state_vec.values.reshape(self.model_obj.original_dim)
            self.model_obj.generate(x0=gridded_values, n_steps=n_steps)
            new_vals = self.model_obj.values

            new_vec = dab.vector.StateVector(values=new_vals, store_as_jax=True)

            return new_vec

        def _forecast_x0(self, x0, n_steps):
            self.model_obj.generate(x0=x0.reshape(self.model_obj.original_dim),
                                    n_steps=n_steps)
            return self.model_obj.values

        def compute_tlm(self, state_vec, n_steps):
            x0 = state_vec.values
            return jax.jacrev(self._forecast_x0, argnums=0)(x0, n_steps), self._forecast_x0(x0, n_steps)

    fc_model = PyQGModel(model_obj=model_pyqg)

    
    ### Set up DA matrices: H (observation), R (obs error), B (background error)
    sigma_obs=sigma_obs_multiplier*obs_sd[obs_vec_pyqg.location_indices[0]]
    sigma_bg = sigma_bg_multiplier*obs_sd
    H = np.zeros((obs_location_count, nature_run.system_dim))
    H[np.arange(H.shape[0]), obs_vec_pyqg.location_indices[0]] = 1
    R = (sigma_obs**2) * np.identity(obs_location_count)
    B = (sigma_bg**2)*np.identity(nature_run.system_dim)
    
    
    ### Prep DA    
    dc = dab.dacycler.Var4DBackprop(
        system_dim=nature_run.system_dim,
        delta_t=nr_eval.delta_t,
        H=H,
        B=B,
        R=R,
        # These will be set later in run_backprop_4dvar
        num_epochs=None, 
        learning_rate=None,
        lr_decay=None,
        model_obj=fc_model,
        obs_window_indices=[0,3,6],
        steps_per_window=7, # 7 instead of 6 because inclusive of 0 and 6
        )
    
    # Generate initial conditions
    cur_tstep = 0
    x0_original = nr_eval.values[cur_tstep] + np_rng.normal(size=(nature_run.system_dim,),
                                                            scale=sigma_bg)
    x0_sv = dab.vector.StateVector(
        values=x0_original,
        store_as_jax=True)
    start_time = nr_eval.times[cur_tstep]
    
    # Return necessary objects for running DA with different LR/LR Decay
    return dc, x0_sv, start_time, obs_vec_pyqg, nr_eval, sigma_obs

In [1]:

def run_backprop_4dvar(config, dc, x0_sv, start_time, obs_vec, nr_eval,
                       sigma_obs, analysis_window, analysis_time_in_window,
                       num_epochs):
    
    dc.learning_rate = config['lr']
    dc.lr_decay = config['lr_decay']
    dc.num_epochs = num_epochs
    
    try: 
        out_statevec = dc.cycle(
            input_state = x0_sv,
            start_time = start_time,
            obs_vector = obs_vec,
            analysis_window=analysis_window,
            timesteps=int(nr_eval.time_dim/6) - 2,
            obs_error_sd=sigma_obs,
            analysis_time_in_window=analysis_time_in_window)

        rmse = np.sqrt(np.mean(np.square(
            nr_eval.values[:out_statevec.values.shape[0]] - out_statevec.values
        )))
        train.report({'rmse':rmse})
        
    # Catch problem with exploding gradients
    except jaxlib.xla_extension.XlaRuntimeError:
        train.report({'rmse':999999})
    
    return 

# Hyperparameter optimization on validation set

Using RayTune to optimize alpha (learning rate) and alpha decay

In [6]:
space = {
    "lr": hp.loguniform("lr", -10, 0),
    "lr_decay": hp.uniform("lr_decay", 0.1, 0.99),
}



In [7]:
%%time

all_results_df_list = []
system_dim_xy_list = [16, 24, 32]

for system_dim_xy in system_dim_xy_list:
    tune_time_start = timer()

    random_seed = system_dim_xy
    
    # Run prep
    run_dict = dict(
        system_dim_xy=system_dim_xy, 
        nr_steps=nr_steps,
        spinup_size=spinup_size,
        valid_size=valid_size,
        test_size=test_size,
        test_run=False,
        delta_t=delta_t,
        sigma_bg_multiplier=0.5,
        sigma_obs_multiplier=1.25,
        random_seed=random_seed)
    dc, x0_sv, start_time, obs_vec, nr_eval, sigma_obs = prep_backprop_4dvar(**run_dict)
    
    
    print('Starting... {} system dim'.format(system_dim_xy))
    trainable_with_system_dim = tune.with_parameters(
        run_backprop_4dvar, 
        dc=dc,
        x0_sv=x0_sv,
        start_time=start_time,
        obs_vec=obs_vec,
        nr_eval=nr_eval,
        sigma_obs=sigma_obs, 
        analysis_window=analysis_window,
        analysis_time_in_window=analysis_time_in_window,
        num_epochs=num_epochs
    )
    
    # Set CPU requirement to num cpus on your machine to NOT run in parallel
    # in order to get a better time comparison. Reduce this to run in parallel
    # and speed up the process.
    trainable_with_resources = tune.with_resources(trainable_with_system_dim, {"cpu": 16})
    hyperopt_search = HyperOptSearch(space, metric="rmse", mode="min")
    
    # Run tuner
    tuner = tune.Tuner(
        trainable_with_resources,
        tune_config=tune.TuneConfig(
            num_samples=50,
            search_alg=hyperopt_search,
        ),
    )
    results = tuner.fit()

    cur_results_df = results.get_dataframe()

    cur_results_df['system_dim_xy'] = system_dim_xy
    
    tune_time = timer() - tune_time_start
    
    cur_results_df['total_tune_time'] = tune_time

    all_results_df_list.append(cur_results_df)

(run_backprop_4dvar pid=907508) 2024-03-25 22:00:22.538342: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=907508) 2024-03-25 22:00:22.970447: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=907508) 2024-03-25 22:00:23.083461: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=907508) 2024-03-25 22:00:26.246313: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none
(run_backprop_4dvar pid=907508) 2024-03-25 22:00:26.296158: E external/org_tensorflow/tensorflow/compiler/xla/python/pjit.cc:461] fastpath_data is none


(run_backprop_4dvar pid=907508) [[2221.41596682 2183.85257707 2175.2235743 ]
(run_backprop_4dvar pid=907508)  [2173.01625192 2134.58696133 2125.7590487 ]
(run_backprop_4dvar pid=907508)  [2136.60414203 2099.90330435 2091.4825808 ]
(run_backprop_4dvar pid=907508)  [2410.10002074 2362.12413619 2351.0899557 ]
(run_backprop_4dvar pid=907508)  [2457.61213631 2410.06449095 2399.1392089 ]
(run_backprop_4dvar pid=907508)  [2329.42541594 2282.78466374 2272.0634738 ]
(run_backprop_4dvar pid=907508)  [2499.69977601 2446.16737891 2433.8724011 ]
(run_backprop_4dvar pid=907508)  [2683.91305304 2622.91824781 2608.8878787 ]
(run_backprop_4dvar pid=907508)  [2727.92786552 2662.19174647 2647.0755678 ]
(run_backprop_4dvar pid=907508)  [2789.38912044 2719.7306154  2703.7191148 ]
(run_backprop_4dvar pid=907508)  [2818.6376595  2751.58649937 2736.1605504 ]
(run_backprop_4dvar pid=907508)  [2932.26601373 2861.95681994 2845.7782725 ]
(run_backprop_4dvar pid=907508)  [3228.82297731 3144.95616779 3125.6711332 ]

(run_backprop_4dvar pid=907508) [[ 2221.41596682  2221.16099743  2221.04622127]
(run_backprop_4dvar pid=907508)  [ 2202.82741775  2202.55292362  2202.42935851]
(run_backprop_4dvar pid=907508)  [ 2166.81132249  2166.55081168  2166.43354177]
(run_backprop_4dvar pid=907508)  [ 2124.07377144  2123.80257025  2123.6804884 ]
(run_backprop_4dvar pid=907508)  [ 2256.0632236   2255.78388874  2255.65814561]
(run_backprop_4dvar pid=907508)  [ 2270.23078726  2269.93913119  2269.80784364]
(run_backprop_4dvar pid=907508)  [ 2262.37710274  2262.0670631   2261.92749943]
(run_backprop_4dvar pid=907508)  [ 2283.42185917  2283.09557466  2282.94869921]
(run_backprop_4dvar pid=907508)  [ 2332.52740957  2332.1836802   2332.02895185]
(run_backprop_4dvar pid=907508)  [ 2484.2875439   2483.90356861  2483.73072425]
(run_backprop_4dvar pid=907508)  [ 2539.34479673  2538.94379352  2538.76328409]
(run_backprop_4dvar pid=907508)  [ 2538.04416479  2537.64459681  2537.46473388]
(run_backprop_4dvar pid=907508)  [ 2593.

(run_backprop_4dvar pid=907508) [[ 2221.41596682  2221.34626173  2221.32090729]
(run_backprop_4dvar pid=907508)  [ 2203.01443686  2202.93937049  2202.91206596]
(run_backprop_4dvar pid=907508)  [ 2167.13959137  2167.068329    2167.04240817]
(run_backprop_4dvar pid=907508)  [ 2124.5219813   2124.44777994  2124.4207901 ]
(run_backprop_4dvar pid=907508)  [ 2256.64962702  2256.57318487  2256.54537997]
(run_backprop_4dvar pid=907508)  [ 2271.00906383  2270.92922499  2270.90018471]
(run_backprop_4dvar pid=907508)  [ 2263.3876474   2263.30275115  2263.27187125]
(run_backprop_4dvar pid=907508)  [ 2284.75531936  2284.66593573  2284.63342366]
(run_backprop_4dvar pid=907508)  [ 2334.28169377  2334.18748466  2334.15321738]
(run_backprop_4dvar pid=907508)  [ 2486.44648976  2486.34122215  2486.30293252]
(run_backprop_4dvar pid=907508)  [ 2541.98946096  2541.87947294  2541.83946632]
(run_backprop_4dvar pid=907508)  [ 2541.16259565  2541.05293867  2541.0130525 ]
(run_backprop_4dvar pid=907508)  [ 2597.

(run_backprop_4dvar pid=907508) [[ 2221.41596682  2221.22231022  2221.18576776]
(run_backprop_4dvar pid=907508)  [ 2202.93613415  2202.72761003  2202.68826217]
(run_backprop_4dvar pid=907508)  [ 2167.00212321  2166.80418946  2166.76684008]
(run_backprop_4dvar pid=907508)  [ 2124.33424777  2124.12816796  2124.08928153]
(run_backprop_4dvar pid=907508)  [ 2256.40396649  2256.19168124  2256.15162389]
(run_backprop_4dvar pid=907508)  [ 2270.68296796  2270.46127901  2270.41944768]
(run_backprop_4dvar pid=907508)  [ 2262.96415583  2262.728453    2262.68397714]
(run_backprop_4dvar pid=907508)  [ 2284.19641953  2283.94830372  2283.90148579]
(run_backprop_4dvar pid=907508)  [ 2333.54629421  2333.28483772  2333.23550244]
(run_backprop_4dvar pid=907508)  [ 2485.54127822  2485.24916388  2485.19404376]
(run_backprop_4dvar pid=907508)  [ 2540.88036597  2540.57521333  2540.51763296]
(run_backprop_4dvar pid=907508)  [ 2539.85447382  2539.55031255  2539.49291935]
(run_backprop_4dvar pid=907508)  [ 2595.

...

(run_backprop_4dvar pid=907508)  [2048.61343053 2005.1362226  1989.688648  ]
(run_backprop_4dvar pid=907508)  [2138.29417645 2093.53745005 2077.6377193 ]
(run_backprop_4dvar pid=907508)  [2262.05326906 2206.68460097 2186.9927816 ]
(run_backprop_4dvar pid=907508)  [2334.63705426 2276.4226063  2255.6583807 ]
(run_backprop_4dvar pid=907508)  [2368.21646529 2301.91438181 2278.341712  ]
(run_backprop_4dvar pid=907508)  [2348.80959939 2290.29374858 2269.4299609 ]
(run_backprop_4dvar pid=907508)  [2488.02447775 2419.7510311  2395.3995972 ]
(run_backprop_4dvar pid=907508)  [2475.67129911 2403.68781523 2378.095082  ]
(run_backprop_4dvar pid=907508)  [2548.68479299 2475.75132352 2449.8236648 ]
(run_backprop_4dvar pid=907508)  [2647.65510026 2573.52011056 2547.0549159 ]
(run_backprop_4dvar pid=907508)  [2739.03441008 2660.77515318 2632.8307144 ]
(run_backprop_4dvar pid=907508)  [2899.95453501 2809.10893678 2776.6597125 ]
(run_backprop_4dvar pid=907508)  [3022.54338511 2922.064231   2886.2853168 ]

(run_backprop_4dvar pid=907508)  [2010.27020344 1920.84346853 1908.2007546 ]
(run_backprop_4dvar pid=907508)  [1955.61550928 1874.8476123  1863.391852  ]
(run_backprop_4dvar pid=907508)  [1950.43758968 1871.72525478 1860.7926258 ]
(run_backprop_4dvar pid=907508)  [2100.42508906 2010.49279375 1997.0233751 ]
(run_backprop_4dvar pid=907508)  [2000.22307271 1913.98800686 1902.1552306 ]
(run_backprop_4dvar pid=907508)  [2122.90229001 2026.01642815 2012.732523  ]
(run_backprop_4dvar pid=907508)  [2063.37784826 1973.94484765 1961.4490218 ]
(run_backprop_4dvar pid=907508)  [2063.09520537 1976.62119975 1963.9584168 ]
(run_backprop_4dvar pid=907508)  [2074.79303739 1983.21064364 1970.4460282 ]
(run_backprop_4dvar pid=907508)  [2052.01878033 1964.57407222 1951.9428144 ]
(run_backprop_4dvar pid=907508)  [2050.80393459 1959.84292261 1946.5751282 ]
(run_backprop_4dvar pid=907508)  [2083.85520151 2003.44428085 1992.3842819 ]
(run_backprop_4dvar pid=907508)  [2108.38809932 2015.73573141 2002.9275884 ]

(run_backprop_4dvar pid=907508) [[ 2221.41596682  2219.71899953  2219.47289899]
(run_backprop_4dvar pid=907508)  [ 2201.90745283  2200.08335236  2199.81881755]
(run_backprop_4dvar pid=907508)  [ 2165.19959662  2163.47081351  2163.22010859]
(run_backprop_4dvar pid=907508)  [ 2121.8776681   2120.07968137  2119.81894549]
(run_backprop_4dvar pid=907508)  [ 2253.19528311  2251.34519212  2251.07690174]
(run_backprop_4dvar pid=907508)  [ 2266.43073789  2264.50200079  2264.2223314 ]
(run_backprop_4dvar pid=907508)  [ 2257.45188682  2255.40451467  2255.10763187]
(run_backprop_4dvar pid=907508)  [ 2276.93259241  2274.78255926  2274.47080031]
(run_backprop_4dvar pid=907508)  [ 2324.00393571  2321.74441337  2321.41677553]
(run_backprop_4dvar pid=907508)  [ 2473.81858388  2471.29773347  2470.93220889]
(run_backprop_4dvar pid=907508)  [ 2526.54741697  2523.92083167  2523.53997455]
(run_backprop_4dvar pid=907508)  [ 2522.99410522  2520.38414682  2520.00570585]
(run_backprop_4dvar pid=907508)  [ 2574.

2024-03-25 22:32:24,650	INFO tune.py:1047 -- Total run time: 1929.41 seconds (1928.66 seconds for the tuning loop).


CPU times: user 1min 29s, sys: 15.6 s, total: 1min 45s
Wall time: 52min 31s


In [8]:
full_results_df = pd.concat(all_results_df_list)
full_results_df.to_csv('./out/pyqg_jax/pyqg_jax_raytune_sgdopt_werrors_v13_hessian_approx_3epochs_system_dims_16_24_32.csv')